In [35]:
VERSION = 'test'

%load_ext tensorboard

from model.model import *
from process_func import *
import tensorflow as tf
DATA_DIR = f'datasets/'
map = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Other', 
       'Col': 'Other', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Other', 
       'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
       'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
titles = ['Miss', 'Mr', 'Mrs', 'Royal', 'Other', 'Master']

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [36]:
data = Titanic(DATA_DIR, show_head = False)
data.Preprocess(map, titles)
dataset = data.GetXandY()

Initializing...
Data Loaded.
Data Preprocessing...
Done Preprocessing.
Returned Data Dictionary


In [37]:
print(dataset['x_test'].shape, dataset['x_train'].shape)
print(dataset['x_train'][0])
print(dataset['y_train'][0])
print(dataset['x_test'][0])
print(dataset['y_test'][0])

(418, 15) (891, 15)
[ 1.         0.         0.         0.         1.         0.
  0.         1.         0.         0.         0.         0.
  1.         0.        -0.5031762]
0.0
[ 1.         0.         0.         0.         1.         0.
  0.         1.         0.         0.         0.         0.
  1.         0.        -0.5031762]
2.0


In [38]:
model = BinaryClassification(dataset['x_train'].shape)

Model: "TitanicBC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 15)]              0         
                                                                 
 Dense0 (Dense)              (None, 13)                208       
                                                                 
 dropout_3 (Dropout)         (None, 13)                0         
                                                                 
 Dense1 (Dense)              (None, 8)                 112       
                                                                 
 Output (Dense)              (None, 1)                 9         
                                                                 
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [39]:
BATCH_SIZE = 64
EPOCHS = 100
CP_dir = SetCheckpoint(VERSION)
TB_dir = SetLog(VERSION)
callbacks = DefCallbacks(VERSION, CP = CP_dir, TB = TB_dir)
model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks, 
          validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 2)

Epoch 1/100
12/12 - 0s - loss: 0.7777 - binary_accuracy: 0.3652 - val_loss: 0.7696 - val_binary_accuracy: 0.2458 - 428ms/epoch - 36ms/step
Epoch 2/100
12/12 - 0s - loss: 0.7431 - binary_accuracy: 0.3708 - val_loss: 0.7191 - val_binary_accuracy: 0.5419 - 109ms/epoch - 9ms/step
Epoch 3/100
12/12 - 0s - loss: 0.6878 - binary_accuracy: 0.5562 - val_loss: 0.6684 - val_binary_accuracy: 0.5698 - 149ms/epoch - 12ms/step
Epoch 4/100
12/12 - 0s - loss: 0.6540 - binary_accuracy: 0.6180 - val_loss: 0.6191 - val_binary_accuracy: 0.6983 - 128ms/epoch - 11ms/step
Epoch 5/100
12/12 - 0s - loss: 0.6210 - binary_accuracy: 0.6840 - val_loss: 0.5764 - val_binary_accuracy: 0.7542 - 115ms/epoch - 10ms/step
Epoch 6/100
12/12 - 0s - loss: 0.6009 - binary_accuracy: 0.6966 - val_loss: 0.5447 - val_binary_accuracy: 0.8045 - 121ms/epoch - 10ms/step
Epoch 7/100
12/12 - 0s - loss: 0.5772 - binary_accuracy: 0.7416 - val_loss: 0.5169 - val_binary_accuracy: 0.8324 - 84ms/epoch - 7ms/step
Epoch 8/100
12/12 - 0s - loss:

In [40]:
model.load_weights(CP_dir)
pred = model.predict(dataset['x_test'])
pred = tf.reshape(pred, (-1))
data_check =  pd.read_csv(DATA_DIR + "gender_submission.csv")
pred_series = pd.Series(pred)
print(pred_series)
# Creating new column of predictions in data_check dataframe
data_check['check'] = pred_series
series = []
for val in data_check.check:
    if val >= 0.5:
        series.append(1)
    else:
        series.append(0)
data_check['final'] = series
match = 0
nomatch = 0
for val in data_check.values:
    if val[1] == val[3]:
        match += 1
    else:
        nomatch += 1

print(match/data_check.shape[-2])

14/14 [==============================] - 0s 1ms/step
0      0.094324
1      0.653093
2      0.136294
3      0.114805
4      0.896592
         ...   
413    0.092267
414    0.872662
415    0.094553
416    0.103984
417    0.793211
Length: 418, dtype: float32
0.8875598086124402


In [41]:
temp = pd.DataFrame(pd.read_csv(DATA_DIR + "test.csv")['PassengerId'])
temp['Survived'] = data_check['final']
temp.to_csv(f"submission/submission_{VERSION}.csv", index = False)

In [42]:
# Family size from Family Name
# apply scaler on Age, Fare?
# Title (Mr. Miss. ) - Married?
# Social class from Ticket class and Fare